In [1]:
# Third test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
# Global Setup
# Creating temp local repos to test on

TMP=/tmp/temp-git-test


git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"

# Move to second local to push some changes
command cd "$TMP" 
git pull origin main 
echo "hello second local" > hello.txt
git add hello.txt
git commit -m "first commit on second local"
git push


Cloning into '/tmp/temp-git-test'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 485 (delta 46), reused 48 (delta 14), pack-reused 405
Receiving objects: 100% (485/485), 42.31 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (283/283), done.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
[main c5b743e] first commit on second local
 1 file changed, 1 insertion(+)
 create mode 100644 hello.txt
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 4 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 310 bytes | 310.00 KiB/s, done.
Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   b783454..c5b743e  main -> main


In [3]:
# git-diff-plus tests
cat hello.txt > bye.txt
echo "differ" >> hello.txt
diffs=$(git-diff-plus hello.txt bye.txt | grep "^\+\w" | wc -l)


In [4]:
# Assert 
# there should only be one line of difference
[ $diffs -eq 1 ]; echo $?;

0


In [5]:
# git-diff-list tests
changed=$(git-diff-list | wc -l)

In [6]:
# Assert 
# there should be one file changed
[ $changed -eq 1 ]; echo $?;

0


In [7]:
# git-vdiff-alias tests
killall nvim > /dev/null
git-vdiff-alias 

instances=$(ps -e | grep "nvim" | wc -l)

nvim: no process found
/dev/null: no process found
[1] 14839

[1]+  Stopped                 git-difftool-plus "$@"


In [8]:
# Assert there is an instance of nvim 
[ $instances -eq 1 ]; echo $?;

1


In [9]:
# git-next-checkin tests
rm -f bye.txt
previous_changes=$( git status -s | wc -l )

removed 'bye.txt'


In [10]:
export GIT_MESSAGE="checkin"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
git-next-checkin hello.txt
rm -rf $(ls | grep '[.][0-9]*[.]')
changes=$( git status -s | wc -l )

issuing: (sleep 5; git-difftool-plus "hello.txt") &
[2] 14913
TODO: modify the GIT_MESSAGE (escaping $'s, etc.) and verify read OK in commit confirmation.
GIT_NO_CONFIRM set to 1, so skipping confirmation
Not zipping changes because PRESERVE_GIT_STASH not 1
issuing: git stash
issuing: git pull --all
issuing: git stash pop
Not unzipping changes (because PRESERVE_GIT_STASH not 1)
	modified:   hello.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	_git-diff-list.14695.bash
	_git-diff.14695.list

no changes added to commit (use "git add" and/or "git commit -a")
Dropped refs/stash@{0} (19986fb93a6c396d0112e0c9d6a40c87d8adafd6)

issuing: git add hello.txt

GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git commit -m 'default'
    [main c9cc662] default
     1 file changed, 1 insertion(+)
GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git push --verbose
Pushing to https://github.com/LorenzoFerraces/bash-test.git
Enumerating object

In [11]:
# Assert there were changes before, and no changes now
[ $previous_changes -gt 1 ]; echo $?;
[ $changes -eq 0 ]; echo $?;

0
0


In [12]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null
rm -rf $(ls | grep '[.][0-9]*[.]')
git rm -f *.txt > /dev/null
git add . > /dev/null
git commit -m "cleanup test files"
git push

[main 9f44c53] cleanup test files
 1 file changed, 2 deletions(-)
 delete mode 100644 hello.txt
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 4 threads
Compressing objects: 100% (1/1), done.
Writing objects: 100% (2/2), 243 bytes | 243.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   c9cc662..9f44c53  main -> main


In [13]:
# delete test local repos

command cd "/tmp"
rm -rf temp-git-test > /dev/null
export GIT_NO_CONFIRM=$previous
